In [ ]:
from huggingface_hub import login
from decouple import config

huggingface_token = config('HUGGINGFACE_TOKEN')
login(token=huggingface_token, add_to_git_credential=True)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = "LGAI-EXAONE/EXAONE-3.5-32B-Instruct-AWQ"
model_orin = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj"]
)

# 모델에 LoRA 어댑터 추가
model = get_peft_model(model_orin, peft_config)

print(model.print_trainable_parameters())
print(model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 92,274,688 || all params: 1,141,511,168 || trainable%: 8.0836
None
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ExaoneForCausalLM(
      (transformer): ExaoneModel(
        (wte): Embedding(102400, 5120, padding_idx=0)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-63): 64 x ExaoneBlock(
            (ln_1): ExaoneRMSNorm()
            (attn): ExaoneAttention(
              (attention): ExaoneSdpaAttention(
                (rotary): ExaoneRotaryEmbedding()
                (k_proj): lora.AwqLoraLinear(
                  (base_layer): WQLinear_GEMM(in_features=5120, out_features=1024, bias=False, w_bit=4, group_size=128)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=5120, out_features=64, bias=False)
                  )
                  (lora_B): Mo

In [3]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/mnt/ssd/1/sanguk/dataset/iljoo_expanded_responses_dataset.jsonl")

In [4]:
def format_sample(sample):
    messages = [
        {"role": "user", "content": sample["instruction"]},
        {"role": "assistant", "content": sample["response"]}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False, max_length=1024)

# 데이터셋에 "text" 컬럼 추가
dataset = dataset.map(lambda x: {"text": format_sample(x)})

# 데이터셋을 8:2로 분할 (train: 80%, test: 20%)
split_dataset = dataset["train"].train_test_split(test_size=0.2, shuffle=True, seed=42)

# 분할된 데이터셋 확인
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 데이터셋 크기 출력 (확인용)
print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"평가 데이터셋 크기: {len(eval_dataset)}")

학습 데이터셋 크기: 4892
평가 데이터셋 크기: 1223


In [5]:
import os
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import EarlyStoppingCallback
from peft import PeftModel

hub_path = "/mnt/ssd/1/hub"
save_model_path = os.path.join(hub_path, "models-iljoodeephub-LGAI-EXAONE-32B_AWQ_fp16_test3")

training_args = TrainingArguments(
    output_dir=save_model_path,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    fp16=True,
    save_strategy="epoch",
    save_total_limit=1,
    eval_strategy="epoch",     
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_strategy="epoch", 
    logging_first_step=True,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  
    processing_class=tokenizer,
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=10,        
        early_stopping_threshold=0.01     
    )],
)

trainer.train()

# 평가
metrics = trainer.evaluate()
print(metrics)

# 최적 체크포인트 경로를 기준으로 병합 및 저장
if trainer.state.best_model_checkpoint:  # 최적 체크포인트가 존재하는 경우
    best_checkpoint_path = trainer.state.best_model_checkpoint
    trainer.save_model(best_checkpoint_path)
    print(f"Merged model saved to {best_checkpoint_path}")
else:
    print("No best checkpoint found.")

Epoch,Training Loss,Validation Loss
1,1.596100,1.179814


{'eval_loss': 1.1798144578933716, 'eval_runtime': 147.3617, 'eval_samples_per_second': 8.299, 'eval_steps_per_second': 1.038, 'epoch': 1.0}
Merged model saved to /mnt/ssd/1/hub/models-iljoodeephub-LGAI-EXAONE-32B_AWQ_fp16_test3/checkpoint-306
